Solve various ODEs at
- https://github.com/JuliaLang/julialang.github.com/blob/317db891aa70e038138e154806426bdedd2b4ccb/blog/_posts/2017-10-13-gsoc-NeuralNetDiffEq.md

In [2]:
using Plots
using DifferentialEquations
include("NN_solver.jl")

predict (generic function with 2 methods)

In [90]:
default(size = (300, 200)) # plot size

# Single ODE

## ODE definition

In [83]:
f1 = (t,u) -> (t^3 + 2*t + (t^2)*((1+3*(t^2))/(1+t+(t^3))) - u*(t + ((1+3*(t^2))/(1+t+t^3))))

function f1!(t, y, dy)
    dy[1] = f1.(t, y[1])
end

y0_list = [0.5]
tspan = (0.0,1.0)

(0.0, 1.0)

## Standard solver

In [84]:
prob = ODEProblem(f1!, y0_list, tspan)
sol = solve(prob, saveat=0.05, reltol=1e-6, abstol=1e-6);

## ANN solver

In [85]:
t = collect(linspace(tspan[1], tspan[2], 11))
t = reshape(t, 1, :)

1×11 Array{Float64,2}:
 0.0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.0

In [86]:
nn = init_nn(f1!, t, y0_list);
show(nn)

Neural ODE Solver
Number of equations:       1
Initial condition y0:      [0.5]
Numnber of hidden units:   10
Number of training points: 11


In [87]:
reset_weights!(nn)
@time train!(nn, iterations=200, show_every=50)

Iter     Function value   Gradient norm 
     0     2.623957e+01     2.067160e+01
    50     3.028757e-06     3.089499e-04
   100     1.206001e-08     5.582171e-05
   150     1.077311e-09     9.921641e-07
  0.962489 seconds (1.49 M allocations: 207.273 MiB, 6.58% gc time)


1-element Array{Any,1}:
 Any[[-0.552385; 0.833365; … ; 2.12217; -1.01152], [-1.20315, 0.805016, -0.801564, -1.23397, 1.94078, -0.732993, -0.766307, -1.5494, 1.10752, 1.14787], [0.680237 0.193621 … 0.647421 -0.0215278], [-0.147552]]

## Compare results

In [88]:
y_pred_list, _ = predict(nn)

(Any[[0.5 0.461856 … 0.936847 1.10109]], Any[[-0.499993 -0.267939 … 1.52037 1.76412]])

In [91]:
plot(sol.t, sol[1,:], label="True", legend = :topleft)
plot!(nn.t[:], y_pred_list[1][:], label="NN solver", 
     lw=0, marker=:circle, markerstrokewidth = 0, markersize=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 1.1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 True 
 
 
 
 
 NN solver

# ODE system: Lotka Volterra

## Problem definition

In [92]:
function lotka_volterra!(t, u, du)
    du[1] = 1.5 .* u[1] - 1.0 .* u[1].*u[2]
    du[2] = -3 .* u[2] + u[1].*u[2]
end

y0_list = [1.0, 1.0]
tspan = (0.0,5.0)

(0.0, 5.0)

## Standard solver

In [96]:
prob = ODEProblem(lotka_volterra!, y0_list, tspan)
sol = solve(prob, saveat=0.05, reltol=1e-6, abstol=1e-6);

## ANN solver

In [119]:
t = collect(linspace(tspan[1], tspan[2], 101))
t = reshape(t, 1, :)

1×101 Array{Float64,2}:
 0.0  0.05  0.1  0.15  0.2  0.25  0.3  …  4.75  4.8  4.85  4.9  4.95  5.0

In [120]:
nn = init_nn(lotka_volterra!, t, y0_list, n_hidden = 20);
show(nn)

Neural ODE Solver
Number of equations:       2
Initial condition y0:      [1.0, 1.0]
Numnber of hidden units:   20
Number of training points: 101


In [121]:
reset_weights!(nn)
@time train!(nn, show_every=100)

Iter     Function value   Gradient norm 
     0     3.233682e+04     3.344732e+05
   100     9.723398e-02     3.021397e-01
   200     1.715799e-02     8.127844e-01
   300     6.739010e-03     8.081406e-02
   400     4.016431e-03     3.566961e-02
   500     3.025833e-03     8.330645e-02
 69.678518 seconds (29.94 M allocations: 54.071 GiB, 14.85% gc time)


2-element Array{Any,1}:
 Any[[-2.68926; -0.912137; … ; 2.11675; 3.90474], [-2.73917, 5.33023, -1.19236, 3.45574, 2.62952, -0.167461, -1.29285, 2.39335, -4.89233, 0.261155, -1.29872, -7.31569, -6.26058, -2.57676, 1.57567, 8.53934, -1.55495, -2.78594, -4.69763, -8.4033], [1.20642 -1.06455 … -2.92624 4.1368], [2.05461]]    
 Any[[0.115523; 1.73919; … ; 0.82579; -0.685205], [1.5372, -3.98838, -0.820788, 0.779029, -0.526023, -0.214606, 3.59275, 2.15521, -0.984073, 3.03974, 6.21031, -1.84196, -1.47078, 2.1272, 9.41779, 0.401007, -0.254896, 2.38234, -1.1771, 0.817692], [0.939779 -3.26831 … -2.28549 -2.09149], [-2.02568]]

In [127]:
train!(nn, show_every=100)

Iter     Function value   Gradient norm 
     0     3.340465e-04     8.889990e-02
   100     2.432191e-04     2.342532e-02
   200     1.630592e-04     1.354503e-01
   300     9.679183e-05     2.382050e-02
   400     6.365625e-05     1.475263e-02
   500     4.314416e-05     2.799422e-02


2-element Array{Any,1}:
 Any[[-2.59604; -2.07659; … ; 1.16725; 3.24193], [-3.11282, 10.5805, -0.00354115, 3.70254, 2.64118, -0.917079, -1.54316, 5.14621, -4.13396, 0.759862, -1.68842, -7.55587, -5.57965, -4.8299, 1.90214, 6.36085, -1.32899, -3.09408, -2.12824, -7.31495], [1.33957 -0.552222 … -2.53282 3.40068], [1.78877]]
 Any[[0.18383; 1.58429; … ; 0.408338; -0.460286], [1.36974, -3.4527, -1.15789, 0.370143, -0.658553, 0.102475, 10.4988, 2.16825, -1.15125, 3.52585, 6.64828, -1.78345, -1.24216, 2.15075, 8.4809, 0.176388, -0.284706, 5.22562, -1.57384, 2.37337], [1.11684 -4.11094 … -2.09169 -2.82755], [-1.93418]]    

## Compare results

In [128]:
y_pred_list, _ = predict(nn)

(Any[[1.0 1.02803 … 5.82479 6.0965], [1.0 0.905441 … 0.54572 0.632703]], Any[[0.510378 0.611422 … 5.55038 5.29278], [-2.00017 -1.78554 … 1.54638 1.94706]])

In [129]:
plot(sol.t, sol[1,:], label="y1 true", legend = :topleft)
plot!(sol.t, sol[2,:], label="y2 true")

plot!(nn.t[:], y_pred_list[1][:], label="y1 NN", 
     lw=0, marker=:circle, markerstrokewidth = 0, markersize=3)
plot!(nn.t[:], y_pred_list[2][:], label="y2 NN", 
     lw=0, marker=:circle, markerstrokewidth = 0, markersize=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 true 
 
 
 
 y2 true 
 
 
 
 
 y1 NN 
 
 
 
 
 y2 NN